In [58]:
import os 


In [59]:
os.chdir(r"D:\SAMITH\Github\wine-quality-predictor")

In [60]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    """Data Ingestion Configuration"""
    root_dir: Path
    source_URL: str 
    local_data_file: Path

In [61]:
from wine_quality_predictor.constants import *
from wine_quality_predictor.utils.common import read_yaml, create_directories

In [62]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        """Method to get the data ingestion configuration"""
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
        )
        return data_ingestion_config

In [63]:
import os
import urllib.request as request
from wine_quality_predictor import logger
from wine_quality_predictor.utils.common import get_size

In [68]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
    
        local_path = Path(self.config.local_data_file)

        # Ensure the parent directory exists
        local_path.parent.mkdir(parents=True, exist_ok=True)

        if not local_path.exists():
            try:
                filename, headers = request.urlretrieve(
                    url=self.config.source_URL,
                    filename=str(local_path)
                )
                logger.info(f"{filename} downloaded with headers: \n{headers}")
                return filename
            except Exception as e:
                logger.error(f"Failed to download file: {e}")
                raise
        else:
            logger.info(f"File already exists: {local_path}")
            return str(local_path)

In [69]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()


except Exception as e:
    raise e

[2025-04-30 14:43:11,700: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-30 14:43:11,702: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-30 14:43:11,704: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-04-30 14:43:11,706: INFO: common: created directory at: artifacts]
[2025-04-30 14:43:11,707: INFO: common: created directory at: artifacts/data_ingestion]
[2025-04-30 14:43:12,268: INFO: 3398899755: artifacts\data_ingestion\data\winequalityN.csv downloaded with headers: 
Connection: close
Content-Length: 390376
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: text/plain; charset=utf-8
ETag: "1d18ba2fbc422055a785a5e2d840e47343b337eb2a52b8c915ca44a1a779b03f"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: A582:4A5A8:14128:2557C:6811FD76
Accept-Ranges: byte